### Import Libraries

In [31]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [32]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [35]:
df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [36]:
df2.head(40)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avenge

In [37]:
df2.drop('title',axis=1,inplace=True)

In [38]:
df1.columns = ['id', 'title', 'cast', 'crew']
df = df2.merge(df1,on='id')

In [39]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Demographic filtering model

We'll use the IMDb movie rating formula. 

The formula for calculating the Top Rated 250 Titles gives a true Bayesian estimate: 

weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C 

where: 

* R = average for the movie (mean) = (Rating) 

* v = number of votes for the movie = (votes)

* C = Mean vote across whole report

* m = minimum number of votes required to be listed

In [40]:
C = df['vote_average'].mean()
C

6.092171559442011

In [41]:
m = df['vote_count'].quantile(0.85)

m

1300.6999999999998

In [42]:
dff = df.copy()

In [43]:
df = df.loc[df['vote_count']>=m]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721 entries, 0 to 4602
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                721 non-null    int64  
 1   genres                721 non-null    object 
 2   homepage              480 non-null    object 
 3   id                    721 non-null    int64  
 4   keywords              721 non-null    object 
 5   original_language     721 non-null    object 
 6   original_title        721 non-null    object 
 7   overview              721 non-null    object 
 8   popularity            721 non-null    float64
 9   production_companies  721 non-null    object 
 10  production_countries  721 non-null    object 
 11  release_date          721 non-null    object 
 12  revenue               721 non-null    int64  
 13  runtime               721 non-null    float64
 14  spoken_languages      721 non-null    object 
 15  status                

In [44]:
# weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C

def wtdrt(x,m=m,C=C):
    V = x['vote_count']
    R = x['vote_average']
    
    return (R*(V/(V+m)) + (C*(m/(V+m))))

In [45]:
df['Score'] = df.apply(wtdrt,axis=1)

In [46]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count,title,cast,crew,Score
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",7.090010
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",6.718859
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",6.253124
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",7.411541
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",6.097027


In [47]:
df = df.sort_values('Score',ascending=False)
df['original_title'].head(10)

1881                         The Shawshank Redemption
662                                        Fight Club
3232                                     Pulp Fiction
65                                    The Dark Knight
3337                                    The Godfather
96                                          Inception
809                                      Forrest Gump
95                                       Interstellar
329     The Lord of the Rings: The Return of the King
1990                          The Empire Strikes Back
Name: original_title, dtype: object


### Drawbacks

* Too generic

* Recommendations can be personalized

## Moving to Content based filtering

In [48]:
df1.columns = ['id', 'title', 'cast', 'crew']
df = df2.merge(df1,on='id')

In [49]:
df['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [50]:
df['overview'][1]

'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.'

In [51]:
df['overview'][4]

"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands."

### Natural Language Processing


#### Count Vectorizer

#### TF-IDF - Term Frequency Inverse Document Frequency vectorizer

##### Helps us to find the overall importance of the word in a set of documents.

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

df['overview'] = df['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['overview'])

tfidf_matrix.shape

(4803, 20978)

In [53]:
df.shape

(4803, 22)

In [54]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Let's create a similarity matrix

In [55]:
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [56]:
df_sim = pd.DataFrame(cosine_sim)

df_sim.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.000000,0.000000,0.024995,0.000000,0.030353,0.000000,0.037581,0.000000,0.000000,0.000000,0.019788,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.034970,0.047055,0.000000,0.025185,0.000000,0.061534,0.000000,0.0,0.0,0.000000,0.074546,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.029175,0.042176,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.033369,0.000000,0.000000,0.022676,0.000000,0.000000,0.000000,0.000000,0.050839,0.000000,0.036632,0.007443,0.000000,0.032567,0.008340,0.018219,0.006358,0.000000,0.0,0.000000,0.000000,0.005118,0.036645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.013252,0.021844,0.0,0.025478,0.000000,...,0.019201,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.009379,0.012083,0.013650,0.018175,0.010143,0.000000,0.018211,0.008614,0.018056,0.006907,0.009857,0.006438,0.000000,0.000000,0.0,0.0,0.015251,0.000000,0.000000,0.000000,0.006895,0.000000,0.000000,0.000000,0.0,0.021605,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.030949,0.024830,0.000000,0.000000,0.165090,0.000000,0.000000,0.000000,0.000000,0.000000,0.052149,0.033391,0.000000,0.000000,0.000000,0.0,0.018329,0.000000,0.000000,0.000000,0.000000,0.000000,0.134112,0.000000,0.000000,0.000000,0.0,0.0,0.075706,0.015518,0.0,0.013552,0.000000,...,0.017459,0.000000,0.000000,0.0,0.063869,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.024927,0.000000,0.000000,0.000000,0.000000,0.019865,0.000000,0.000000,0.000000,0.000000,0.028601,0.0,0.0,0.000000,0.000000,0.027695,0.000000,0.000000,0.000000,0.017768,0.000000,0.0,0.014882,0.000000,0.000000
3,0.024995,0.000000,0.000000,1.000000,0.010433,0.005145,0.012601,0.026954,0.020652,0.133740,0.000000,0.000000,0.000000,0.000000,0.000000,0.004071,0.021121,0.000000,0.006769,0.010765,0.007178,0.033381,0.0,0.000000,0.019238,0.017013,0.018846,0.000000,0.008351,0.025822,0.017550,0.029948,0.021709,0.0,0.0,0.000000,0.028418,0.0,0.030632,0.010885,...,0.013738,0.000000,0.022601,0.0,0.043093,0.000000,0.006721,0.012392,0.00000,0.020640,0.000000,0.0,0.000000,0.000000,0.005540,0.000000,0.000000,0.000000,0.058285,0.000000,0.011572,0.000000,0.000000,0.021169,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033864,0.042752,0.022692
4,0.000000,0.033369,0.000000,0.010433,1.000000,0.000000,0.009339,0.037407,0.000000,0.017148,0.020154,0.000000,0.031781,0.000000,0.010713,0.000000,0.045257,0.020359,0.016908,0.016255,0.009145,0.017275,0.0,0.000000,0.016511,0.000000,0.053875,0.052310,0.025467,0.016102,0.000000,0.011121,0.021416,0.0,0.0,0.015827,0.038104,0.0,0.000000,0.021240,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.012721,0.000000,0.0,0.000000,0.000000,0.005220,0.000000,0.000000,0.000000,0.000000,0.000000,0.017890,0.009415,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.012730,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.006126,0.000000,0.000000
5,0.030353,0.000000,0.000000,0.005145,0.000000,1.000000,0.000000,0.000000,0.000000,0.006209,0.016412,0.000000,0.000000,0.017483,0.023126,0.040659,0.000000,0.008836,0.000000,0.000000,0.222847,0.016675,0.0,0.000000,0.008141,0.024812,0.016754,0.032883,0.000000,0.017186,0.246472,0.000000,

In [57]:
indices = pd.Series(df.index,index=df['title'])

indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [58]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    # get index of movie title
    idx = indices[title]
    
    # get list of cosine similarities
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    
    # convert it into a list of tuples looking 
    # like this >>> (index of movie, similarity score)
    # sort the list basis the similarity score in descending order
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
    
    
    # get top 10 elements of list
    sim_scores = sim_scores[1:11]
    
    # get movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    
    # return top 10
    return df['title'].iloc[movie_indices]

In [59]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [60]:
get_recommendations('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title, dtype: object

### The  second recommender engine looks better than the first one

### Problems : recommender system is too tied up with overview column and is giving recommendations in a tight space.

In [61]:
df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [62]:
df['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [63]:
df['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [64]:
df['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [65]:
features = ['cast','crew','keywords','genres']

from ast import literal_eval #abstract syntax trees

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [66]:
df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [67]:
df['crew'][0]

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [68]:
df[features]

,cast,crew,keywords,genres
0,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
1,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
3,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
...,...,...,...,...
4798,"[{'cast_id': 1, 'character': 'El Mariachi', 'c...","[{'credit_id': '52fe44eec3a36847f80b280b', 'de...","[{'id': 5616, 'name': 'united states–mexico ba...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4799,"[{'cast_id': 1, 'character': 'Buzzy', 'credit_...","[{'credit_id': '52fe487dc3a368484e0fb013', 'de...",[],"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
4800,"[{'cast_id': 8, 'character': 'Oliver O’Toole',...","[{'credit_id': '52fe4df3c3a36847f8275ecf', 'de...","[{'id': 248, 'name': 'date'}, {'id': 699, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4801,"[{'cast_id': 3, 'character': 'Sam', 'credit_id...","[{'credit_id': '52fe4ad9c3a368484e16a36b', 'de...",[],[]


In [69]:
df[features].to_csv('featurelist.csv')

### Features - HOMEWORK

* 3 top actors

* Director

* Related genres and plot keywords

# 1. Write a function to extract director from crew column

In [70]:
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

In [71]:
get_director(df['crew'][95])

'Christopher Nolan'

In [72]:
df[df['title']=='Interstellar']

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count,title,cast,crew
95,165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",http://www.interstellarmovie.net/,157336,"[{'id': 83, 'name': 'saving the world'}, {'id'...",en,Interstellar,Interstellar chronicles the adventures of a gr...,724.247784,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2014-11-05,675120017,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Mankind was born on Earth. It was never meant ...,8.1,10867,Interstellar,"[{'cast_id': 9, 'character': 'Joseph Cooper', ...","[{'credit_id': '52fe4bbf9251416c910e4801', 'de..."


# 2. Top 3 values of actor, genre, and crew - whatever has length > 3 pick first 3

In [73]:
def get_top_three(x):
    if isinstance(x,list):
        names = [i['name'] for i in x]
        
        # checking if there are more than 3 elements
        if len(names)>3:
            names = names[:3]
            
        return names
    return []

In [74]:
df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [75]:
get_top_three(df['genres'][0])

['Action', 'Adventure', 'Fantasy']

In [76]:
df['director'] = df['crew'].apply(get_director)

In [77]:
features = ['cast','genres','keywords']

for feature in features:
    df[feature] = df[feature].apply(get_top_three)

In [78]:
df[features]

,cast,genres,keywords
0,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[Action, Adventure, Fantasy]","[culture clash, future, space war]"
1,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island]"
2,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[Action, Adventure, Crime]","[spy, based on novel, secret agent]"
3,"[Christian Bale, Michael Caine, Gary Oldman]","[Action, Crime, Drama]","[dc comics, crime fighter, terrorist]"
4,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion]"
...,...,...,...
4798,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms]"
4799,"[Edward Burns, Kerry Bishé, Marsha Dietlein]","[Comedy, Romance]",[]
4800,"[Eric Mabius, Kristin Booth, Crystal Lowe]","[Comedy, Drama, Romance]","[date, love at first sight, narration]"
4801,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[],[]


# 3. Clean the data - replace punctuations, and spaces

In [79]:
def cleanData(x):
    
    if isinstance(x,list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x,str):
            return str.lower(x.replace(' ',''))
        else:
            return []

In [80]:
features = ['director','cast','genres','keywords']
df[features]

,director,cast,genres,keywords
0,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[Action, Adventure, Fantasy]","[culture clash, future, space war]"
1,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island]"
2,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[Action, Adventure, Crime]","[spy, based on novel, secret agent]"
3,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman]","[Action, Crime, Drama]","[dc comics, crime fighter, terrorist]"
4,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion]"
...,...,...,...,...
4798,Robert Rodriguez,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms]"
4799,Edward Burns,"[Edward Burns, Kerry Bishé, Marsha Dietlein]","[Comedy, Romance]",[]
4800,Scott Smith,"[Eric Mabius, Kristin Booth, Crystal Lowe]","[Comedy, Drama, Romance]","[date, love at first sight, narration]"
4801,Daniel Hsia,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[],[]


In [81]:
for feature in features:
    df[feature] = df[feature].apply(cleanData)

In [82]:
df[features]

,director,cast,genres,keywords
0,jamescameron,"[samworthington, zoesaldana, sigourneyweaver]","[action, adventure, fantasy]","[cultureclash, future, spacewar]"
1,goreverbinski,"[johnnydepp, orlandobloom, keiraknightley]","[adventure, fantasy, action]","[ocean, drugabuse, exoticisland]"
2,sammendes,"[danielcraig, christophwaltz, léaseydoux]","[action, adventure, crime]","[spy, basedonnovel, secretagent]"
3,christophernolan,"[christianbale, michaelcaine, garyoldman]","[action, crime, drama]","[dccomics, crimefighter, terrorist]"
4,andrewstanton,"[taylorkitsch, lynncollins, samanthamorton]","[action, adventure, sciencefiction]","[basedonnovel, mars, medallion]"
...,...,...,...,...
4798,robertrodriguez,"[carlosgallardo, jaimedehoyos, petermarquardt]","[action, crime, thriller]","[unitedstates–mexicobarrier, legs, arms]"
4799,edwardburns,"[edwardburns, kerrybishé, marshadietlein]","[comedy, romance]",[]
4800,scottsmith,"[ericmabius, kristinbooth, crystallowe]","[comedy, drama, romance]","[date, loveatfirstsight, narration]"
4801,danielhsia,"[danielhenney, elizacoupe, billpaxton]",[],[]


In [83]:
def joinKW(x):
    return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+ ''.join(x['director'])+' '+' '.join(x['genres'])

In [84]:
df['joined'] = df.apply(joinKW,axis=1)

In [85]:
df['joined'][0]

'cultureclash future spacewar samworthington zoesaldana sigourneyweaver jamescameron action adventure fantasy'

# 4. Merge the data and vectorize

In [86]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')

cm = cv.fit_transform(df['joined'])

In [87]:
cm.shape

(4803, 11520)

# 5. Create new similarity matrix using linear_kernel like above

In [88]:
cosine_sim2 = cosine_similarity(cm,cm)

In [89]:
df_cs = pd.DataFrame(cosine_sim2)
df_cs.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.0,0.3,0.2,0.1,0.2,0.3,0.0,0.2,0.2,0.3,0.3,0.2,0.3,0.2,0.3,0.2,0.2,0.3,0.1,0.3,0.3,0.210819,0.210819,0.210819,0.2,0.1,0.2,0.2,0.2,0.2,0.286039,0.2,0.2,0.2,0.0,0.2,0.2,0.2,0.3,0.2,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.129099,0.000000,0.0,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.095346,0.0,0.0,0.0,0.0
1,0.3,1.0,0.2,0.1,0.2,0.3,0.0,0.2,0.2,0.3,0.3,0.2,0.7,0.4,0.3,0.2,0.2,0.4,0.1,0.3,0.3,0.210819,0.210819,0.210819,0.2,0.0,0.2,0.2,0.2,0.2,0.286039,0.2,0.3,0.2,0.0,0.2,0.2,0.2,0.3,0.2,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.129099,0.000000,0.0,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.095346,0.0,0.0,0.0,0.0
2,0.2,0.2,1.0,0.2,0.3,0.2,0.0,0.2,0.1,0.2,0.2,0.4,0.2,0.2,0.2,0.2,0.2,0.2,0.1,0.2,0.2,0.210819,0.105409,0.210819,0.2,0.0,0.2,0.2,0.2,0.6,0.190693,0.2,0.2,0.2,0.0,0.2,0.2,0.1,0.2,0.2,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.129099,0.000000,0.0,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.119523,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.1,0.000000,0.0,0.000000,0.000000,0.0,0.190693,0.0,0.0,0.0,0.0
3,0.1,0.1,0.2,1.0,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.1,0.1,0.2,0.0,0.1,0.1,0.1,0.1,0.1,0.105409,0.000000,0.000000,0.2,0.1,0.1,0.1,0.1,0.1,0.095346,0.2,0.0,0.1,0.0,0.1,0.1,0.0,0.1,0.1,...,0.0,0.119523,0.0,0.0,0.0,0.119523,0.129099,0.095346,0.0,0.2,0.0,0.105409,0.119523,0.105409,0.141421,0.3,0.0,0.239046,0.0,0.105409,0.0,0.111803,0.141421,0.0,0.000000,0.1,0.111803,0.129099,0.0,0.1,0.111803,0.0,0.105409,0.091287,0.0,0.190693,0.0,0.1,0.0,0.0
4,0.2,0.2,0.3,0.1,1.0,0.2,0.0,0.3,0.1,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.3,0.2,0.2,0.2,0.2,0.210819,0.105409,0.105409,0.2,0.0,0.3,0.3,0.3,0.2,0.190693,0.3,0.2,0.3,0.0,0.3,0.3,0.1,0.2,0.3,...,0.0,0.119523,0.0,0.0,0.0,0.000000,0.129099,0.000000,0.0,0.1,0.0,0.000000,0.000000,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.105409,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.091287,0.0,0.095346,0.0,0.0,0.0,0.0


In [90]:
df = df.reset_index()

indices = pd.Series(df.index,index=df['title'])

In [91]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [92]:
get_recommendations('The Dark Knight Rises',cosine_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

In [93]:
get_recommendations('The Avengers',cosine_sim2)

7                  Avengers: Age of Ultron
26              Captain America: Civil War
79                              Iron Man 2
169     Captain America: The First Avenger
174                    The Incredible Hulk
85     Captain America: The Winter Soldier
31                              Iron Man 3
33                   X-Men: The Last Stand
68                                Iron Man
94                 Guardians of the Galaxy
Name: title, dtype: object

In [94]:
get_recommendations('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title, dtype: object

# 6. in get_recommendation function pass the movie title, and new similarity matrix

In [95]:
df['genres'][0]

['action', 'adventure', 'fantasy']

# Collaborative Filtering

1. **User Based Collaborative Filtering**

    * Recommends products to a user that another similar user has liked.

    * *Issues*
        * User preferences change over time
        * Similarity matrix is pre-computed so changes in user preferences for the uncovered window will not be accounted for.


2. **Item Based Collaborative Filtering**

    * Instead of creating similarity matrix between users, we'll measure the similarity between the items that **targeted user** has rated.


* Scalability : One of the issues in recommender systems is giving real time recommendations. So optimization is a concerned area.


* As # of users (m) and number of products (n) increase, the time complexity will increase as well. O(m * n)


* Sparse matrix is generated.

## Singular Value Decomposition

#### Metrics : RMSE - the lower the better

#### Library : Surprise

In [96]:
!pip install surprise

     |████████████████████████████████| 11.8MB 9.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617551 sha256=f951da90640d3b47b069a80b60d285df4e73c9b7eb96c0eb8b223009f41cb11f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [99]:
from surprise import Reader, Dataset, SVD

from surprise.model_selection import cross_validate

reader = Reader()

dfd = pd.read_csv('ratings_small.csv')

In [101]:
dfd.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [102]:
data = Dataset.load_from_df(dfd[['userId', 'movieId', 'rating']],reader)

algo = SVD()

cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8997  0.8994  0.8972  0.8916  0.9009  0.8977  0.0033  
MAE (testset)     0.6930  0.6905  0.6883  0.6881  0.6935  0.6907  0.0023  
Fit time          5.06    5.03    5.07    5.05    5.26    5.10    0.08    
Test time         0.19    0.17    0.27    0.23    0.29    0.23    0.05    


{'fit_time': (5.064715385437012,
  5.03089714050293,
  5.074527263641357,
  5.05324649810791,
  5.255002737045288),
 'test_mae': array([0.69299624, 0.69051729, 0.68826396, 0.68808925, 0.69354038]),
 'test_rmse': array([0.89970337, 0.89940573, 0.89720263, 0.89155773, 0.90087786]),
 'test_time': (0.186295747756958,
  0.16923761367797852,
  0.2710988521575928,
  0.2300872802734375,
  0.28942441940307617)}

In [103]:
dfd[dfd['userId']==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [104]:
algo.predict(1,302,verbose=True)

user: 1          item: 302        r_ui = None   est = 2.73   {'was_impossible': False}


Prediction(uid=1, iid=302, r_ui=None, est=2.734340206722868, details={'was_impossible': False})

In [105]:
algo.predict(1,18,verbose=True)

user: 1          item: 18         r_ui = None   est = 2.51   {'was_impossible': False}


Prediction(uid=1, iid=18, r_ui=None, est=2.5107100026218405, details={'was_impossible': False})